## Using NexMotion API to Control MiniBot
### Pre-requirement:
* Python2.7 32bit version (NexMotion.dll is 32bit version)
* Create MiniBot Configuration from NexMotion Studio (NexMotionNcf.ncf)
* For controlling real MiniBOT, do homing process in NexMotion Studio first.

#### 1. Include Necessary Libarary with environment variable

In [1]:
%config IPCompleter.greedy=True
# include nexmotion library
import nexmotion as nm
from nexmotion.errors import *
from nexmotion.constants import *
import pandas as pd
dll_path = None
ini_path = None

#### 1.5 Set Path for dll and create config ini file
Running simulation with install NexMotion Studio. Make sure you have `NexMotion.dll` and `NexMotionNcf` in `config` folder

In [ ]:
import os
import ctypes
cwd_path = os.getcwd()
# this is necessay dll file for NexMotion Lib
dll_path = cwd_path + '\\config\\NexMotion.dll'
print dll_path
# set the path for ini file
ini_path = cwd_path + '\\config\\NexMotionLibConfig.ini'
print ini_path
# create ini file
#with open(ini_path, 'w') as confile:
#    confile.write('[NexMotion]\n')
#    confile.write('NCF_PATH = \"' + cwd_path + '\\config\\NexMotionNcf.ncf' + '\"')

#### 1.6 Debug Starting steps

In [ ]:
nmc = nm.Control(dll_path=dll_path)

In [ ]:
nmc.setIniPath(ini_path)

In [ ]:
nmc.dll_.NMC_DeviceCreate(nmc.type_, nmc.index_, ctypes.byref(nmc.id_))

In [ ]:
nmc.dll_.NMC_DeviceLoadIniConfig(nmc.id_)

In [ ]:
nmc.dll_.NMC_DeviceStart(nmc.id_)

#### 2. Start Device

In [2]:
if dll_path is None:
    nmc = nm.Control()  # will use default path for dll and ini file
else:
    nmc = nm.Control(dll_path=dll_path)
    nmc.setIniPath(ini_path)
ret = nmc.deviceOpenup(type_=DEVICE_TYPE_ETHERCAT) # DEVICE_TYPE_SIMULATOR for simulation
if ret != nm.errors.SUCCESS:
    print "NMC_DeviceOpenUp error, error code =", ret

Dynamics library version = 10040966 ( 1 , 0 , 4 , 966 )
device id = 1


#### 3. Check Device State
Get the Device State.
Here are the 4 possible state. If nothing wrong, the device state will be **4**

| Name   |      Value      |  Description |
|--------|:-------------:|------:|
| NMC_DEVICE_STATE_INIT      | 1 | Initial |
| NMC_DEVICE_STATE_READY     | 2 | Ready |
| NMC_DEVICE_STATE_ERROR     | 3 | Error |
| NMC_DEVICE_STATE_OPERATION | 4 | Operation |

In [3]:
ret = nmc.deviceGetState()
if ret != SUCCESS:
    print "NMC_DeviceGetState error, error code =", ret
else:
    if nmc.devState_.value != DEVICE_STATE_OPERATION:
        print "device state is not OPERATION" 
    else:
        print "device state =", nmc.devState_.value


device state = 4


#### 4. Display Virtual Robot for visualization

In [4]:
nmc.group3DShow(top_=True)
nmc.group3DDrawPath(enable=True)

0

#### 5. Get device information: Count of group

In [ ]:
nmc.deviceGetGroupCount()
print "count of group index", nmc.numGroup_.value

#### 6. Get device information: Count of Group Axis
For MiniBot, the below code will output: 6

In [ ]:
nmc.deviceGetGroupAxisCount()
print "group index", nmc.index_.value, ": count of group axis = ", nmc.numGroupAxis_.value

#### 7. Clean alarm of drives of each group
When sending movement command (such as moveLine, groupLine), the robot may not execute correctly (such as desired position is outside its workspace or in singular point. Then, the controller will set group state to `GROUP_ERROR_STOP`).
These two are useful command to clean alarm or reset state of robot.

In [ ]:
nmc.groupResetDriveAlmAll()
nmc.groupResetState()

## 8. Check group state
Here are all possible group (movement) state:
Get the state of group. Please refer to the below table for details.

|Value | State | Description | 
|------|:-------:|-------------:| 
| 0 | GROUP_DISABLE     | A group axis is disabled. | 
| 1 | GROUP_STAND_STILL | All group axes are enabled. | 
| 2 | GROUP_STOPPED     | After NMC_GroupStop() is called, the group is stopped. |
| 3 | GROUP_STOPPING    | After NMC_GroupStop() is called, the group is stopping. |
| 4 | GROUP_MOVING      | The group is moving. |
| 5 | GROUP_HOMING      | The group is homing. |
| 6 | GROUP_ERROR_STOP  | An error is occurred in a group axis. |

##### Note: if Group State == 6 (error stop), please run step 7 to reset state.
##### Note: if Group State == 0 (disable), please run step 9 to servo on in order to move robot arm.


In [7]:
ret = nmc.groupGetState()
if ret != SUCCESS:
    print "groupGetState error, error code =", ret
else:
    print "Group State = ", nmc.groupState_.value

Group State =  1


#### 9. Enable all axes of group (Servo On)
MiniBot will make a sound when it is enabled.
And, group sate will be changed (could run step **8** to check again)  
Note: need to Enable before running any movement command

In [6]:
ret = nmc.groupEnable()
if ret != SUCCESS:
    print "NMC_DeviceEnableAll error, error code =", ret
else:
    print "enable all done" 

enable all done


#### 10. Show Actual Joint Position
This step can be called to check actual joint values after you move or home the Minibot.  
For controlling real MiniBOT, make sure you do homing process (or this command will not show correctly)

In [7]:
jntPos = [0.] * 6
ret = nmc.groupGetActualPosAcs(jntPos)
if ret != SUCCESS:
    print "NMC_GroupGetActualPosAcs error, error code =", ret
else:
    print "NMC_GroupGetActualPosAcs:"
    for i in range(0, 6, 1):
        print "   Joint", i ,"val:", '%.3f' % jntPos[i]

NMC_GroupGetActualPosAcs:
   Joint 0 val: 0.019
   Joint 1 val: 89.958
   Joint 2 val: -0.161
   Joint 3 val: -0.038
   Joint 4 val: -60.869
   Joint 5 val: -89.919


## 11. Execute Group PTP motion to a pre-define position
##### Note: Please make sure all the joints are finish homing (if controlling real MiniBOT)

In [10]:
desPos = [0, 45, 0, 0, -45, -90]
nmc.groupPtpAcsAll(desPos)

0

#### 12. Show Actual Cartesian Position 
This step can be called to check actual cartesian values.

In [54]:
#get current cartesian value
cartPos = [0.] * 6
ret = nmc.groupGetActualPosPcs(cartPos)
if ret != SUCCESS:
    print "groupGetActualPosPcs error, error code =", ret
else:
    print "groupGetActualPosPcs:"
    coord_list = ['x', 'y', 'z', 'a', 'b', 'c']
    for i in range(0, 6, 1):
        print "\t", coord_list[i] ,"val:", '%.3f' % cartPos[i]

groupGetActualPosPcs:
	x val: 357.976
	y val: -109.352
	z val: 234.126
	a val: 89.831
	b val: -0.160
	c val: -178.915


## 13. Execute Group Line motion to a pre-define position
##### `Note: Please make sure all the joints are finish homing`

In [53]:
pos = [0.] * 6
nmc.groupGetActualPosPcs(pos)
pos[0] += -1.0; pos[1] += -0; pos[2] += -0
nmc.groupLine(pos)

0

#### 15. record current pose (Joint space and Cartesian space)

In [55]:
nmc.recordPoint()

0

#### 16. show recorded points

In [56]:
df = pd.DataFrame(data=nmc.pnt_list, 
                  columns=['j1','j2','j3','j4','j5','j6','x','y','z','a','b','c'])
df.to_csv("record_pnt.csv")
print df

          j1         j2         j3      j4         j5       j6           x  \
0   0.019091  89.957727  -0.160910 -0.0378 -60.869250  -89.919  323.882949   
1   0.002728  44.982954  -0.017046 -0.0126 -45.007875  -89.982  430.458095   
2  -8.709545  31.208182  11.052272 -0.0522 -43.090875  -98.595  454.565936   
3 -13.397727  28.313864  16.638409 -0.1431 -45.843750 -103.194  459.388273   
4 -16.945909  46.354773 -19.298182 -0.3402 -28.141875 -106.479  357.975932   

            y           z          a         b           c  
0    0.053173  574.965931  89.948332 -0.001380  151.072410  
1    0.005719  293.310290  89.993643  0.008917 -179.958036  
2  -69.695683  235.598554  89.923189 -0.089393 -179.173646  
3 -109.597453  235.119295  89.895941 -0.105057 -179.108890  
4 -109.352911  234.125720  89.833083 -0.159563 -178.915004  


## 17. move PTP like GRC

In [103]:
nmc.movePTP(0)

0

## 18 moveLine like GRC

In [102]:
nmc.moveLine(2)

0

## 19. Halt Robot Movement

In [101]:
nmc.groupHalt()

0

## 20. Set DO value
Use Digital Output to control gripper or suctions.

In [ ]:
DO_list = [[0,4],[]]  # Turn digital oupt 0, 2, 4 to `on` and other to `off`
nmc.writeOutputMemory(DO_list)

### Write Script 

In [104]:
import time
nmc.movePTP(0)
time.sleep(1)
nmc.movePTP(1)
nmc.movePTP(2)
time.sleep(3)
nmc.moveLine(3)
for i in xrange(2):
    nmc.moveLine(4)
    nmc.moveLine(3)
    time.sleep(3)
nmc.moveLine(2)
nmc.moveLine(1)
nmc.movePTP(0)


#nmc.movePTP(0)
#time.sleep(1)
#nmc.movePTP(2)
#time.sleep(1)
#nmc.writeOutputMemory([[2],[]])
#nmc.moveLine(1)
#print nmc.groupGetState()
#print nmc.groupState_.value
#while nmc.groupState_.value == 4:
#    nmc.groupGetState()
##time.sleep(2)
#nmc.writeOutputMemory([[],[]])


0

#### 21. Disable all axes of group (Servo Off)
MiniBot will make a sound when it is disabled.

In [8]:
nmc.groupDisable()

0

#### 22. Shutdown device

In [9]:
nmc.deviceShutdown()

0

### Test http request (Setting)

In [2]:
import requests
import time
import uuid
import base64
# defining the api-endpoint
#API_ENDPOINT = "http://ai.touchcloud.com.tw:5000/api/v1.0/inference"
API_ENDPOINT = "http://192.168.100.200:5000/api/v1.0/inference" 
 
# data to be sent to api
def getDataTemplate():
  data = {
      'UUID': '24ef383c-96ca-4c81-97cd-c3c9515623ef',
      'Thresholds': '0.87',
      'Timestamp': '2018-08-30 12:34:22',
      'Type': 'JPEG',
      'Image': '.......',
      'Mode': 'ALL',
  }
  return data

### Run request for TouchCloud Vision

In [22]:
def run_request():
  # sending post request and saving response as response object
  print(API_ENDPOINT)
  data = getDataTemplate()
  data['UUID'] = str(uuid.uuid4())
  data['Image'] = './test/verify/003417.jpg'
  r = requests.post(url = API_ENDPOINT, json = data)
  #r = requests.get(url = API_ENDPOINT)
  # extracting response text
  pastebin_url = r.text
  result = r.json()
  jpg = result['Image']
  #print(jpg)
    
  image = base64.decodestring(jpg.encode())
  #fh = open("_data/aruco_calib.jpg", "wb")
  fh = open("imageToSave.jpg", "wb")
  fh.write(image)
  fh.close()
  #print(pastebin_url)
  
  result['Image'] = 'Base64String'
  #print(type(result))
  return result

#run_request()

### Get the position and orientation of LED

In [58]:
def get_led_pixel_pose(res, color):
    x_pos, y_pos, ang = 0., 0., 0.
    if color in res:
        x_pos = res[color]['CRX']
        y_pos = res[color]['CRY']
        ang = res[color]['ANGLE']
    else:
        print 'Did not detect ', color, ' led in the plate'
    return x_pos, y_pos, float(ang)


### Convert Pixel Pose to Cartesian Pose with Transformation Matrix after calibration 
Note: MiniBot Need to set Base with Aruco mark that we used for calibration

In [57]:
def pixPos2cartPos(pose, ang,trfMatrix):
    """
    pose: np array (3,1)
    ang: float
    trfMatrix: np Matrix (3,3)
    """
    
    cartPos = trfMatrix.dot(pose)
    print cartPos
    
    cartPos = trfMatrix.dot(pose) / 10
    print cartPos
    # swap x,y
    cartPos[[0,1]] = cartPos[[1,0]]
    if ang > 180:
        ang -= 360
    ang += 90
    return cartPos, ang

In [59]:
import numpy as np
import pickle as pkl


result = run_request()
print '\n\n\n'
print result
x, y, a = get_led_pixel_pose(result, 'red')

print 'LED pose: ', x, y, a
pix_pos = np.array([[x, y, 1.0]]).transpose()
pix_pos.shape

trfM = pkl.load(open('_data/aruco_calib.pkl'))
#print trfM
cartPos, ang = pixPos2cartPos(pix_pos, a, trfM)
print "Cartesian Pose: ", cartPos[0][0], cartPos[1][0], ang

http://192.168.100.200:5000/api/v1.0/inference




{u'blue': {u'Confidence': u'1.0', u'HY': 873, u'ANGLE': u'46.54741809149125', u'BRY': 1048, u'BRX': 1191, u'CRX': 1067, u'CRY': 917, u'HX': 1110, u'TLX': 943, u'TLY': 787}, u'code': u'00', u'UUID': u'823ba14b-6fc5-42c2-a236-ce211a703701', u'color': [u'red', u'yellow', u'green', u'blue'], u'Image': 'Base64String', u'yellow': {u'Confidence': u'0.999023', u'HY': 300, u'ANGLE': u'129.4256989944976', u'BRY': 483, u'BRX': 1247, u'CRX': 1125, u'CRY': 351, u'HX': 1082, u'TLX': 1003, u'TLY': 219}, u'green': {u'Confidence': u'1.0', u'HY': 615, u'ANGLE': u'28.10333164394672', u'BRY': 745, u'BRX': 1484, u'CRX': 1335, u'CRY': 647, u'HX': 1399, u'TLX': 1187, u'TLY': 549}, u'MSG': u'success', u'Type': u'JPEG', u'red': {u'Confidence': u'0.999023', u'HY': 606, u'ANGLE': u'15.880562268420448', u'BRY': 697, u'BRX': 848, u'CRX': 691, u'CRY': 621, u'HX': 758, u'TLX': 535, u'TLY': 546}}
LED pose:  691 621 15.8805622684
[[-272.85022719]
 [ 165.53371774]
 [  